In [ ]:
!wget https://raw.githubusercontent.com/Rexhaif/ner-dialogues-hackathon/master/data/train.conll
!wget https://raw.githubusercontent.com/Rexhaif/ner-dialogues-hackathon/master/data/test.conll

In [1]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
import scipy
import eli5
import pymorphy2
import spacy
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV

In [2]:
nlp = spacy.load("en_core_web_sm")

In [3]:
morph = pymorphy2.MorphAnalyzer()

In [4]:
with open('train.conll') as train_f, open('test.conll') as test_f:
    train_str = train_f.read()
    test_str = test_f.read()

In [5]:
def get_word_pos(word, sent):
    pos = morph.parse(word[0])[0].tag.POS
    if not pos:
        i = sent.index(word)
        sent = ' '.join([tok[0] for tok in sent]).capitalize()
        doc = nlp(sent)
        pos = doc[i].pos_
    return pos

In [6]:
def parse_conll(conll_str):
    sents = [tuple(sent.split()) for sent in conll_str.split('\n\n')]
    splitted_sents = []
    for sent in sents:
        i = iter(sent)
        splitted_sents.append(list(zip(i, i)))
    parsed_sents = [[(word[0], get_word_pos(word, sent), word[1])
                     for word in sent] for sent in splitted_sents]
    return parsed_sents

In [7]:
train_sents = parse_conll(train_str)
test_sents = parse_conll(test_str)

In [56]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [57]:
X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

In [58]:
X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

In [59]:
X_train[0][1]

{'bias': 1.0,
 'word.lower()': 'из',
 'word[-3:]': 'из',
 'word.isupper()': False,
 'word.istitle()': False,
 'word.isdigit()': False,
 'postag': 'PREP',
 'postag[:2]': 'PR',
 '-1:word.lower()': 'спой',
 '-1:word.istitle()': False,
 '-1:word.isupper()': False,
 '-1:postag': 'VERB',
 '-1:postag[:2]': 'VE',
 '+1:word.lower()': 'шоколад',
 '+1:word.istitle()': False,
 '+1:word.isupper()': False,
 '+1:postag': 'NOUN',
 '+1:postag[:2]': 'NO'}

In [60]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=20,
    all_possible_transitions=False,
)
crf.fit(X_train, y_train)

/opt/homebrew/Caskroom/miniforge/base/lib/python3.8/site-packages/sklearn/base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


CRF(algorithm='lbfgs', all_possible_transitions=False, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=20)

In [61]:
eli5.show_weights(crf, top=30)

From \ To,O,B-BOOK,I-BOOK,B-COMPOSER,I-COMPOSER,B-FILM,I-FILM,B-SINGER,I-SINGER,B-SONG,I-SONG
O,0.707,2.192,0.0,2.277,0.0,3.558,0.0,4.117,0.0,5.922,0.0
B-BOOK,0.0,0.0,6.894,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
I-BOOK,0.0,0.0,5.476,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-COMPOSER,-0.176,0.0,0.0,0.0,3.701,0.0,0.0,0.0,0.0,0.0,0.0
I-COMPOSER,0.005,0.0,0.0,0.0,2.87,0.0,0.0,0.0,0.0,0.0,0.0
B-FILM,-0.116,0.0,0.0,0.0,0.0,0.0,6.463,0.0,0.0,0.0,0.0
I-FILM,-0.236,0.0,0.0,0.0,0.0,0.0,5.112,0.0,0.0,0.0,0.0
B-SINGER,-0.528,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.736,0.0,0.0
I-SINGER,0.262,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.537,0.0,0.0
B-SONG,-0.491,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.266


In [62]:
labels = list(crf.classes_)
labels.remove('O')
labels

['B-FILM',
 'B-SONG',
 'B-SINGER',
 'I-SONG',
 'I-SINGER',
 'B-COMPOSER',
 'I-COMPOSER',
 'I-FILM',
 'B-BOOK',
 'I-BOOK']

In [63]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred,
                      average='weighted', labels=labels)

0.6235860877360649

In [64]:
sorted_labels = sorted(
    labels, 
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

              precision    recall  f1-score   support

      B-BOOK      0.320     0.219     0.260        73
      I-BOOK      0.381     0.416     0.398        89
  B-COMPOSER      0.889     0.457     0.604        35
  I-COMPOSER      0.667     0.571     0.615        21
      B-FILM      0.720     0.261     0.383        69
      I-FILM      0.724     0.356     0.477        59
    B-SINGER      0.588     0.901     0.711       253
    I-SINGER      0.581     0.782     0.667       147
      B-SONG      0.831     0.594     0.693       165
      I-SONG      0.865     0.734     0.794       192

   micro avg      0.636     0.636     0.636      1103
   macro avg      0.657     0.529     0.560      1103
weighted avg      0.664     0.636     0.624      1103



/opt/homebrew/Caskroom/miniforge/base/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass labels=['B-BOOK', 'I-BOOK', 'B-COMPOSER', 'I-COMPOSER', 'B-FILM', 'I-FILM', 'B-SINGER', 'I-SINGER', 'B-SONG', 'I-SONG'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


In [67]:
# define fixed parameters and parameters to search
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    max_iterations=100, 
    all_possible_transitions=True
)
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score, 
                        average='weighted', labels=labels)

# search
rs = RandomizedSearchCV(crf, params_space, 
                        cv=10, 
                        verbose=1, 
                        n_jobs=-1, 
                        n_iter=50, 
                        scoring=f1_scorer)
rs.fit(X_train, y_train)

Fitting 10 folds for each of 50 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   24.4s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   59.8s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  1.2min finished


RandomizedSearchCV(cv=10,
                   estimator=CRF(algorithm='lbfgs',
                                 all_possible_transitions=True,
                                 keep_tempfiles=None, max_iterations=100),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'c1': <scipy.stats._distn_infrastructure.rv_frozen object at 0x16b3ecdc0>,
                                        'c2': <scipy.stats._distn_infrastructure.rv_frozen object at 0x1697edca0>},
                   scoring=make_scorer(flat_f1_score, average=weighted, labels=['B-FILM', 'B-SONG', 'B-SINGER', 'I-SONG', 'I-SINGER', 'B-COMPOSER', 'I-COMPOSER', 'I-FILM', 'B-BOOK', 'I-BOOK']),
                   verbose=1)

In [68]:
# crf = rs.best_estimator_
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

best params: {'c1': 0.01790551129101197, 'c2': 0.04208114842093376}
best CV score: 0.7737313371391479
model size: 0.50M


(Лучший результат был примерно 0.776)

In [69]:
crf = rs.best_estimator_
y_pred = crf.predict(X_test)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

              precision    recall  f1-score   support

      B-BOOK      0.533     0.329     0.407        73
      I-BOOK      0.639     0.517     0.571        89
  B-COMPOSER      0.783     0.514     0.621        35
  I-COMPOSER      0.733     0.524     0.611        21
      B-FILM      0.667     0.435     0.526        69
      I-FILM      0.652     0.508     0.571        59
    B-SINGER      0.624     0.846     0.718       253
    I-SINGER      0.611     0.769     0.681       147
      B-SONG      0.770     0.648     0.704       165
      I-SONG      0.849     0.823     0.836       192

   micro avg      0.683     0.681     0.682      1103
   macro avg      0.686     0.591     0.625      1103
weighted avg      0.690     0.681     0.674      1103



In [70]:
from collections import Counter

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))

print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

Top likely transitions:
I-SONG -> I-SONG  6.143864
B-SINGER -> I-SINGER 6.121120
B-SONG -> I-SONG  6.031404
B-BOOK -> I-BOOK  5.534319
I-FILM -> I-FILM  5.445351
B-FILM -> I-FILM  5.182416
I-BOOK -> I-BOOK  5.177909
B-COMPOSER -> I-COMPOSER 4.602690
I-SINGER -> I-SINGER 4.328449
I-COMPOSER -> I-COMPOSER 4.186204
O      -> B-SONG  2.979909
O      -> B-FILM  2.933035
O      -> B-SINGER 1.698378
O      -> B-COMPOSER 1.647821
O      -> B-BOOK  1.547461
I-SONG -> O       0.846811
I-SINGER -> O       0.474327
B-SINGER -> O       0.344661
B-COMPOSER -> O       0.205795
I-FILM -> O       0.002215

Top unlikely transitions:
B-COMPOSER -> I-FILM  -0.890643
B-COMPOSER -> I-BOOK  -0.901831
B-FILM -> I-SINGER -0.945786
B-SONG -> I-FILM  -0.991383
B-COMPOSER -> I-SINGER -1.120498
B-SONG -> I-BOOK  -1.175841
I-BOOK -> O       -1.184475
B-SINGER -> I-BOOK  -1.205242
B-SONG -> I-SINGER -1.210065
B-SINGER -> I-FILM  -1.214210
B-FILM -> I-BOOK  -1.249604
B-SINGER -> I-COMPOSER -1.297584
I-SINGER -> I-SON

In [71]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))    

print("Top positive:")
print_state_features(Counter(crf.state_features_).most_common(30))

print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-30:])

Top positive:
7.366763 B-SINGER -1:word.lower():автора
6.577296 B-SONG   -1:word.lower():песню
5.741748 B-COMPOSER -1:word.lower():композитора
5.253708 B-SINGER -1:word.lower():певца
4.954227 O        BOS
4.544067 B-FILM   -1:word.lower():фильма
4.389317 O        word[-3:]:ора
4.233206 O        word.lower():песню
4.233206 O        word[-3:]:сню
3.945755 B-FILM   -1:word.lower():фильм
3.799074 B-BOOK   -1:word.lower():книгу
3.736182 O        word.lower():композитора
3.550674 B-FILM   word.lower():втор
3.464591 O        word[-3:]:игу
3.310044 B-BOOK   word.lower():разгром
3.145418 B-SONG   -1:word.lower():произведение
3.077304 B-BOOK   word[-3:]:ках
3.025635 O        word.lower():послушаем
3.004001 B-COMPOSER word.lower():листом
2.994592 O        word.lower():дорожку
2.994592 O        word[-3:]:жку
2.982581 B-SONG   -1:word.lower():дорожку
2.898636 O        word.lower():включи
2.898636 O        word[-3:]:ючи
2.868037 B-SINGER word.lower():маикалсан
2.858981 B-FILM   -1:word.lower():из
2.